In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [1]:
from juggling_dlx_milp import *

/home/josue/Documents/git/Jugglyng1/Expérimentations/Juggling DLX/DLX/dlxm.py:69: RuntimeWarning: class "DLX_M::AbstrItem" has no virtual destructor
  class ConcItem(_AbstrItem):


In [2]:
music = [( 1, "do"), ( 2, "do"), ( 3, "do"), 
         ( 4, "re"), ( 5, "mi"), ( 7, "re"), 
         ( 9, "do"), (10, "mi"), (11, "re"),
         (12, "re"), (13, "do")]

In [3]:
balls, throws = music_to_throws(music)

In [4]:
balls

{'do', 'mi', 're'}

In [5]:
throws

[[Throw(ball='do', time=0, max_height=1),
  Throw(ball='re', time=0, max_height=4),
  Throw(ball='mi', time=0, max_height=5)],
 [Throw(ball='do', time=1, max_height=1)],
 [Throw(ball='do', time=2, max_height=1)],
 [Throw(ball='do', time=3, max_height=6)],
 [Throw(ball='re', time=4, max_height=3)],
 [Throw(ball='mi', time=5, max_height=5)],
 [],
 [Throw(ball='re', time=7, max_height=4)],
 [],
 [Throw(ball='do', time=9, max_height=4)],
 [],
 [Throw(ball='re', time=11, max_height=1)],
 []]

### Contraintes

In [6]:
nb_hands = 2
max_height = 5
max_weight = 3
forbidden_multiplex = [(1, 2), (1, 3), (1, 4), (2, )]
multiple_throws = True

### Algorithme

In [7]:
ec_instance1 = throws_to_extended_exact_cover(balls, throws, nb_hands, max_height, max_weight,
                                              forbidden_multiplex, [], multiple_throws)

In [8]:
sol1 = solve_exact_cover_with_milp(ec_instance1, True)

In [9]:
ec_instance2 = throws_to_extended_exact_cover(balls, throws, 2, 4, 2,
                                             [(1, 3)], [(1, 3), (1, 1)], False)

In [10]:
sol2 = solve_exact_cover_with_milp(ec_instance2)

#### Solution en interdisant de lancer plusieurs balles depuis une même main en même temps

In [11]:
print_juggling(sol1)

{'do', 'mi'}    {'re'}    : do -- 1 --> 1
                            re -- 4 --> 1
   {'mi'}       {'do'}    : mi -- 4 --> 1
                            do -- 1 --> 0
   {'do'}         {}      : do -- 1 --> 1
     {}         {'do'}    : 
     {}      {'do', 're'} : re -- 3 --> 0
     {}      {'do', 'mi'} : 
     {}      {'do', 'mi'} : do -- 3 --> 0
   {'re'}       {'mi'}    : mi -- 3 --> ?
   {'re'}         {}      : re -- 3 --> 1
   {'do'}         {}      : 
   {'do'}         {}      : do -- 3 --> ?
     {}         {'re'}    : re -- 1 --> ?
     {}           {}      : 


#### Solution en autorisant de lancer plusieurs balles depuis une même main en même temps

In [12]:
print_juggling(sol2)

{'do', 're'}    {'mi'}    : do -- 1 --> 1
   {'re'}    {'do', 'mi'} : do -- 1 --> 0
{'do', 're'}    {'mi'}    : re -- 2 --> 0
                            do -- 1 --> 1
     {}      {'do', 'mi'} : mi -- 2 --> 0
   {'re'}       {'do'}    : 
{'mi', 're'}    {'do'}    : 
{'mi', 're'}    {'do'}    : re -- 1 --> 1
   {'mi'}    {'do', 're'} : 
   {'mi'}    {'do', 're'} : do -- 1 --> 0
{'do', 'mi'}    {'re'}    : mi -- 1 --> ?
   {'do'}       {'re'}    : re -- 1 --> 0
{'do', 're'}      {}      : re -- 1 --> ?
   {'do'}         {}      : do -- 1 --> ?


#### Même comparaison avec d'autres fonctions d'affichages

In [13]:
print_juggling_solution(sol1)

0 - do : main 0 (0 temps) --> 1 temps
0 - re : main 1 (0 temps) --> 4 temps
0 - mi : main 0 (1 temps) --> 4 temps
1 - do : main 1 (0 temps) --> 1 temps
2 - do : main 0 (0 temps) --> 1 temps
3 - do : main 1 (3 temps) --> 3 temps
4 - re : main 1 (0 temps) --> 3 temps
5 - mi : main 1 (2 temps) --> 3 temps
7 - re : main 0 (1 temps) --> 3 temps
9 - do : main 0 (1 temps) --> 3 temps
11 - re : main 1 (0 temps) --> 1 temps


In [14]:
print_juggling_solution(sol2)

0 - do : main 0 (0 temps) --> 1 temps
0 - re : main 0 (2 temps) --> 2 temps
0 - mi : main 1 (3 temps) --> 2 temps
1 - do : main 1 (0 temps) --> 1 temps
2 - do : main 0 (0 temps) --> 1 temps
3 - do : main 1 (5 temps) --> 1 temps
4 - re : main 0 (2 temps) --> 1 temps
5 - mi : main 0 (4 temps) --> 1 temps
7 - re : main 1 (3 temps) --> 1 temps
9 - do : main 0 (3 temps) --> 1 temps
11 - re : main 0 (0 temps) --> 1 temps


#### Export vers le format du simulateur

In [15]:
juggling_sol_to_simulator(sol1)

[[[('do', 1, 1)],
  [('mi', 1, 4)],
  [('do', 1, 1)],
  [],
  [],
  [],
  [],
  [],
  [('re', 1, 3)],
  [],
  [('do', 1, 3)],
  [],
  [],
  []],
 [[('re', 1, 4)],
  [('do', 0, 1)],
  [],
  [],
  [('re', 0, 3)],
  [],
  [('do', 0, 3)],
  [('mi', 0, 3)],
  [],
  [],
  [],
  [('re', 0, 1)],
  [],
  []]]

### Simulateur

In [27]:
import modele
import ipywidgets
import pythreejs

balls = [{"color":"blue", "tone":"do", "name":"do"},
         {"color":"red", "tone":"re", "name":"re"},
         {"color":"green", "tone":"mi", "name":"mi"},]

# balls = [{"color":"blue", "tone":"do", "name":"do"},
#          {"color":"red", "tone":"re", "name":"re"},
#          {"color":"green", "tone":"mi", "name":"mi"}]

pattern = juggling_sol_to_simulator(sol1)

sides = [-1, 1]

model = modele.Model(balls, pattern)
slider = ipywidgets.FloatSlider(min=0, max=40, step=0.05)
view = modele.View(model, sides)
slider.observe(lambda change: view.update(change['new'], change['old']), names="value")
ipywidgets.VBox([view.widget,slider])

In [30]:
view.widget.camera.position = (0, 0, -50)

In [17]:
dlx = dlx_solver_instance(ec_instance1)

In [18]:
len(dlx.rows)

62

In [19]:
for v in dlx.variables:
    print(v.lower_bound, v.upper_bound, v.secondary)

0 1 False
1 1 False
0 2 False
0 1 False
0 1 False
0 1 False
0 1 False


In [20]:
music1 = [( 1, "do"), ( 2, "do"), ( 3, "do")]

In [21]:
balls1, throws1 = music_to_throws(music1)

In [22]:
balls1, throws1

({'do'},
 [[Throw(ball='do', time=0, max_height=1)],
  [Throw(ball='do', time=1, max_height=1)],
  [Throw(ball='do', time=2, max_height=1)]])

In [23]:
ec_instance3 = throws_to_extended_exact_cover(balls1, throws1, 2, 4, 2, [], [], True)

In [24]:
dlx = dlx_solver_instance(ec_instance3)

In [25]:
cols = []
for i in range(len(dlx.rows)):
    for r in dlx.row_repr(i):
        r1 = r.replace("_", "")
        if r not in cols:
            cols.append(r)
        print(r1, end=" ")
    print("")
for col in cols:
    for v in dlx.variables:
        for k in v:
            if v[k].get_repr() == col:
                print("{}:{}|{}".format(v.lower_bound, v.upper_bound, col.replace("_", "")), end=" ")
                continue
print("")

x0 x1 x2 x3 x4 x5 x6 
x7 x1 x8 x9 x10 x11 x12 
x13 x14 x15 x16 x17 x6 x18 
x19 x14 x20 x21 x22 x12 x23 
x24 x25 x26 x27 x28 x18 x29 
x30 x25 x31 x32 x33 x23 x34 
0:1|x0 1:1|x1 0:1|x2 0:1|x3 0:2|x4 0:1|x5 0:1|x6 0:1|x7 0:1|x8 0:1|x9 0:2|x10 0:1|x11 0:1|x12 0:1|x13 1:1|x14 0:1|x15 0:1|x16 0:2|x17 0:1|x18 0:1|x19 0:1|x20 0:1|x21 0:2|x22 0:1|x23 0:1|x24 1:1|x25 0:1|x26 0:1|x27 0:2|x28 0:1|x29 0:1|x30 0:1|x31 0:1|x32 0:2|x33 0:1|x34 


In [26]:
for i in range(len(dlx.rows)):
    for obj in dlx.row_obj(i):
        print(obj.latex(), end=" ")
    print("")

x_{Throw(ball='do', time=0, max_height=1)}^{0, 1} l_{Throw(ball='do', time=0, max_height=1)} D_{0}^{0} i_{0, 0, 1} w_{0, 0} U_{do, 0}^{0} U_{do, 1}^{0} 
x_{Throw(ball='do', time=0, max_height=1)}^{1, 1} l_{Throw(ball='do', time=0, max_height=1)} D_{0}^{1} i_{0, 1, 1} w_{0, 1} U_{do, 0}^{1} U_{do, 1}^{1} 
x_{Throw(ball='do', time=1, max_height=1)}^{0, 1} l_{Throw(ball='do', time=1, max_height=1)} D_{1}^{0} i_{1, 0, 1} w_{1, 0} U_{do, 1}^{0} U_{do, 2}^{0} 
x_{Throw(ball='do', time=1, max_height=1)}^{1, 1} l_{Throw(ball='do', time=1, max_height=1)} D_{1}^{1} i_{1, 1, 1} w_{1, 1} U_{do, 1}^{1} U_{do, 2}^{1} 
x_{Throw(ball='do', time=2, max_height=1)}^{0, 1} l_{Throw(ball='do', time=2, max_height=1)} D_{2}^{0} i_{2, 0, 1} w_{2, 0} U_{do, 2}^{0} U_{do, 3}^{0} 
x_{Throw(ball='do', time=2, max_height=1)}^{1, 1} l_{Throw(ball='do', time=2, max_height=1)} D_{2}^{1} i_{2, 1, 1} w_{2, 1} U_{do, 2}^{1} U_{do, 3}^{1} 


In [27]:
dlx.all_solutions()

[{0, 3, 4}, {1, 2, 5}]

In [28]:
sols = solve_with_dlx(ec_instance1)

In [29]:
len(sols)

110208

In [30]:
sols[0]

ExactCoverSolution(max_time=13, nb_hands=2, balls={'do', 're', 'mi'}, rows=[[XItem(throw=Throw(ball='do', time=0, max_height=1), hand=0, flying_time=1), LItem(throw=Throw(ball='do', time=0, max_height=1)), DItem(time=0, hand=0), IItem(time=0, hand=0, flying_time=1), MItem(time=0, hand=0, multiplex=(1, 3)), IItem(time=1, hand=0, flying_time=3), IItem(time=1, hand=0, flying_time=1), WItem(time=0, hand=0), UItem(ball='do', time=0, hand=0), UItem(ball='do', time=1, hand=0)], [XItem(throw=Throw(ball='re', time=0, max_height=4), hand=0, flying_time=3), LItem(throw=Throw(ball='re', time=0, max_height=4)), DItem(time=1, hand=0), MItem(time=1, hand=0, multiplex=(1, 3)), WItem(time=0, hand=0), WItem(time=1, hand=0), UItem(ball='re', time=0, hand=0)], [XItem(throw=Throw(ball='mi', time=5, max_height=5), hand=0, flying_time=1), LItem(throw=Throw(ball='mi', time=5, max_height=5)), DItem(time=9, hand=0), MItem(time=9, hand=0, multiplex=(1, 3)), IItem(time=6, hand=0, flying_time=3), IItem(time=6, han

In [31]:
print_juggling_solution(sols[0])

0 - do : main 0 (0 temps) --> 1 temps
0 - re : main 0 (1 temps) --> 3 temps
5 - mi : main 0 (4 temps) --> 1 temps
0 - mi : main 1 (3 temps) --> 2 temps
7 - re : main 1 (3 temps) --> 1 temps
1 - do : main 1 (0 temps) --> 1 temps
2 - do : main 0 (0 temps) --> 1 temps
9 - do : main 0 (3 temps) --> 1 temps
3 - do : main 1 (5 temps) --> 1 temps
11 - re : main 0 (0 temps) --> 1 temps
4 - re : main 0 (2 temps) --> 1 temps


In [32]:
juggling_sol_to_simulator(sols[0])

[[[('do', 1, 1)],
  [('re', 0, 3)],
  [('do', 1, 1)],
  [],
  [],
  [],
  [('re', 1, 1)],
  [],
  [],
  [('mi', 1, 1)],
  [],
  [('re', 1, 1)],
  [('do', 1, 1)],
  []],
 [[],
  [('do', 0, 1)],
  [],
  [('mi', 0, 2)],
  [],
  [],
  [],
  [],
  [('do', 0, 1)],
  [],
  [('re', 0, 1)],
  [],
  [],
  []]]

Une éventuelle contrainte à rajouter : interdire qu'une main ne lance et rattrape une balle au même moment.

...

Après réflexion ce n'est en fait pas nécessaire. Si on veut jongler à la suite 3 lancers de hauteur 1 avec deux mains on va faire : main 0 -> main 1 -> main 0 -> main 1.

Formellement une main relance une balle au même instant où elle la reçoit. En réalité, elle reçoit la balle puis la garde un court instant dans la main (moins d'un temps) puis la relance. Une main ne lance et ne reçoit donc jamais en même temps. Le temps où la balle reste dans la main entre la réception et le lancer par le jongleur fait partie du temps de vol dans le modèle.

## Notes suite à la discussion du 7/05

### Contraintes supplémentaires à ajouter à la réduction
- interdir de lancer deux balles en même temps dans la même main
- score de jouabilité (minimiser le nombre de lancer en même temps)
- temps de départ à prendre en compte

### Corrections à faire
- corriger la réduction pour le cas où on a des lancers multiplex où les balles étaient déjà dans la main (et pas : elles arrivent puis sont relancées immédiatement après)

### Changements à essayer dans l'algorithme
- utiliser l'algo avec les polytopes pour générer toutes les solutions (cf. la solution de cette [question](https://ask.sagemath.org/question/23666/solving-system-of-linear-diophantine-equations/))

### Autres choses pratiques à faire
- améliorer un peu la classe `Audio` (définir à l'initialisation du programme une fonction javascript qui est ensuite simplement appelée par python)

In [20]:
# des erreurs vont être affichées mais c'est bon en fait
generate_full_table(ec_instance)

Rc files read:
  NONE
Latexmk: This is Latexmk, John Collins, 29 September 2020, version: 4.70b.
Rule 'pdflatex': File changes, etc:
   Changed files, or newly in use since previous run(s):
      '/home/josue/Documents/git/Jugglyng1/Expérimentations/full_table.tex'
      'full_table.tex'
------------
Run number 1 of rule 'pdflatex'
------------
------------
Running 'pdflatex  --interaction=nonstopmode -recorder  "/home/josue/Documents/git/Jugglyng1/Expérimentations/full_table.tex"'
------------
Latexmk: applying rule 'pdflatex'...
This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020/Arch Linux) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(/home/josue/Documents/git/Jugglyng1/Expérimentations/full_table.tex
LaTeX2e <2020-10-01> patch level 2
L3 programming layer <2020-12-03> xparse <2020-03-03>
(/usr/share/texmf-dist/tex/latex/standalone/standalone.cls
Document Class: standalone 2018/03/26 v1.3a Class to compile TeX sub-files stan
dalone
(/us

CalledProcessError: Command '['latexmk', '--pdf', '--interaction=nonstopmode', '/home/josue/Documents/git/Jugglyng1/Expérimentations/full_table.tex']' returned non-zero exit status 12.

In [21]:
generate_table(ec_instance)